# Autocomplete in Databricks notebooks
When we use `load_table()` on Databricks, it also offers autocomplete on the column names. No more looking at `df.columns` every minute to remember the column names!

To illustrate this, let us first generate a table that we'll write to the table `person_table`.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.Builder().config("spark.ui.showConsoleProgress", "false").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

23/06/29 09:41:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import pandas as pd

(
    spark.createDataFrame(
        pd.DataFrame(
            dict(
                name=["Jack", "John", "Jane"],
                age=[20, 30, 40],
            )
        )
    )
    .createOrReplaceTempView("person_table")
)


We can now load these data using `load_table()`. Note that the `Schema` is inferred: it doesn't need to have been serialized using `typedspark`.

In [3]:
from typedspark import load_table

df, Person = load_table(spark, "person_table")

You can now use `df` and `Person` just like you would in your IDE, including autocomplete!

In [4]:
(
    df
    .filter(Person.age > 25)
    .show()
)

+----+---+
|name|age|
+----+---+
|John| 30|
|Jane| 40|
+----+---+



Of note, `load_table()` automatically runs `register_schema_to_dataset()` on the resulting `DataSet` and `Schema`, hence resolving potential column disambiguities.

The above has been tested on Databricks, but may also work in other types of notebooks. Best way to find out is to try!

Next to using the Schema in your next query, there can also be reasons you want the code representation of the Schema.

In [5]:
Person

To use this to define the schema in your codebase, the Schema name needs to be changed. One can do this by defining the `schema_name` in the `load_table` function.

In [6]:
df, Person = load_table(spark, "person_table", schema_name="Person")
Person